In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'  # default is ‘last_expr'

%load_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.append('/home/gramener/CameraTraps')  # append this repo to PYTHONPATH

In [ ]:
import json
import os
from collections import Counter, defaultdict
from random import sample
import math

from tqdm import tqdm

from data_management.megadb.schema import sequences_schema_check
from data_management.annotations.add_bounding_boxes_to_megadb import *
from data_management.megadb.converters.cct_to_megadb import make_cct_embedded, process_sequences, write_json

In [ ]:
path_to_output_public = '/home/gramener/ena24_megadb/ena24_megadb_public.json'
path_to_output_private = '/home/gramener/ena24_megadb/ena24_megadb_private.json'

In [ ]:
img_files = os.listdir('/mnt/blobfuse/wildlifeblobssc/images/')

In [ ]:
dataset_name = 'ena24'

In [ ]:
label_map = {'human': 'person', 'vehicle': 'vehicle'}

In [ ]:
# path to the CCT json, or a loaded json object
path_to_image_cct = '/home/gramener/ena24_megadb/ena24.json'  # set to None if not available
path_to_bbox_cct = '/home/gramener/ena24_megadb/ena24.json'  # set to None if not available
assert not (path_to_image_cct is None and path_to_bbox_cct is None)

In [ ]:
embedded = make_cct_embedded(image_db=path_to_image_cct, bbox_db=path_to_bbox_cct)

In [ ]:
sequences = process_sequences(embedded, dataset_name)

In [ ]:
for index, entry in enumerate(sequences):
    for prop in ['id', 'bbox', 'class']:
        if prop == 'bbox':
            for pos, ele in enumerate(entry[prop]):
                ele['species'] = ele['category']
                ele['category'] = label_map.get(ele['category'], 'animal')
                entry[prop][pos] = ele
        sequences[index]['images'][0][prop] = entry[prop]
        del entry[prop]
    sequences[index] = entry

In [ ]:
for each in sequences:
    if (each['images'][0]['file'] in img_files):
        public.append(each)
    else:
        each['dataset'] = 'ena24_private'
        private.append(each)

In [ ]:
sequences_schema_check.sequences_schema_check(public)

In [ ]:
sequences_schema_check.sequences_schema_check(private)

In [ ]:
write_json(path_to_output_public, public)
write_json(path_to_output_private, private)